# MICRO-452 Basics of mobile robotics

## Project Report

Team  13:

Wilhelm Laetitia,
Beuret Sylvain,
Labbe Victor,
Jaffal Oussama

### Table of Contents

1. [Introduction](#introduction)

2. [Global Set-up](#globalsetup)
    * [Map Set-up](#mapsetup)
    * [Libraries & constants](#libraries)
    * [Launch demo](#demo)
    * [Main.py structure](#main) 

3. [Computer vision](#computervision)
    * [Map detection](#mapdetection)
    * [ArUcO detection](#arucodetection)
    * [Obstacles detection](#obstaclesdetection)
    * [Results](#CVresults)
  

4. [Path planning](#pathplanning)
    * [Global Navigation](#globalnavigation)
    * [A* algorithm](#Aalgorithm)
    * [Implementation](#implementation)
    * [Results](#PPresults)

5. [Kalmann Filtering](#filtering)
    * [General, filtering](#genfiltering)
    * [Mathematical model](#mathmodel)
    * [Filter blabla](#filterblabla)
    * [Results](#Kresults)

6. [Motion control](#motioncontrol)
    * [Orientation & angle error](#orientation)
    * [PD controler](#controler)
    * [Compute velocity](#velocity)
    * [Local navigation](#localnavigation)
    * [Results](#MCresults)

7. [Conclusion](#conclusion)

### Introduction <a class="anchor" id="introduction"></a>


This project is made in the frame of the EPFL course: Basics of Mobile Robotics (BoMR). The goal is to program a robot, the Thymio, to go from a specific point to another while avoiding the obstacles and surviving some kidnapping sessions. In order to achieve this, we used computer vision, filtering, global navigation and local navigation.

As you can see, we are on a car racing circuit, where our beloved thymio is racing against the clock.

Our program uses an overhead camera to detect the obstacles, the track, the thymio and the finishing line. The global navigation part, anaibles to do path planning to reach the line (goal). Finally we have a Kalman filter, using the robot's poistion from the camera and odometry, allowing the thymio to know and correct its position in real time, even with a hidden camera.

<div style="text-align: center;">
    <img src="image\schema-explication.png" alt="Structure" title="Map" style="width:600px;"/>
</div>


<div style="text-align:center;">
     "Project structure"
</div>

<div style="text-align: center;">
<img src="image/thymio_futur.jpeg" alt="Setup Map" title="Map" style="width:200px;"/>
</div>


<div style="text-align:center;">
     Our thymio in the futur by AI
</div>

AJOUTER NOTRE VIDEO

from IPython.display import Video

video_path = 'chemin/vers/votre/video.mp4'
video = Video(video_path)

#affiche la vidéo
video

When the camera is on, different options can be displayed, by pressing the following keys: 
* p: Quits the program.

The camera requires around one minute to launch. The terminal will show you the state of the process.

-------

## Global set-up <a class="anchor" id="computervision"></a>

### Map Set-up <a class="anchor" id="setup"></a>

The road is in white and the obstacles are in black. We have in addition some white 3D obstacles for local avoidance, not on the picture. The elbow obstacle is turned 180° to suit our mood at the time. The demo works both ways.



<div style="text-align: center;">
<img src="image/setup_map.jpg" alt="Setup Map" title="Map" style="width:400px;"/> 
</div>


<div style="text-align:center;">
     Map
</div>

### Libraries & Constants<a class="anchor" id="libraries"></a>


Using Numpy, OpenCv and Shapely libraries, we get a livestream from the webcam and we use the 20th frame of the video to detect most of the things that we need to plan the shortest path for the robot.

In [1]:
from timeit import default_timer as timer
from tdmclient import ClientAsync, aw
import time
import cv2
import cv2 as cv
import threading
import keyboard

In [13]:
## Constants
DEFAULT_GRID_RES = 19 #resolution of the grid in pixels
TIMESTEP = 0.075 #time between each iteration of the main loop
SIZE_THYM = 2.5  #size of thymio in number of grid
SPEEDCONV = 0.05
BASESPEED = 50
LOST_TRESH = 6 #treshold to be considered lost
REACH_TRESH = 3 #treshhold to reach current checkpoint
REACH_GOAL_TRESH = 1 #treshhold to reach current checkpoint
GLOBAL_PLANNING = True
GOAL_REACHED = False


<font color="red">### EXPLAIN MEANING OF THE CONSTANTS###</font>

We implemented several classes, so we can use the same variables along the entire code coming from different persons.

### Launch demonstration <a class="anchor" id="setup"></a>

To run our project, you need to download this [GitHub repository](https://github.com/Thebestvic07/BOMR_Project.git).

To launch our demo, you need to follow those steps:
1. Run the main.py file
2. Don't put anything on the map except of the obstacles
3. When "searching thymio and goal" is written in the terminal, you can add both
4. A new window will show the optimal path to follow
4. Press "p" to exist the demo

### Main.py structure <a class="anchor" id="setup"></a>

We decided to use the different following thread in the main so several task can run in parallel.


##### 1. Camera Thread (<span style="text-decoration: underline;">*camera_thread*</span>):

**Function**: run_camera

**Purpose & Purpose & Execution**: The run_camera function is executed in this thread. It continuously captures frames, processes ArUco markers, and updates the robot and goal positions.

##### 2. Thymio Thread (<span style="text-decoration: underline;">*thymio_thread*</span>):
    
**Function**: update_thymio

**Purpose & Execution**: The update_thymio function is executed in this thread. It continuously ($f=10s^{-1}$) reads variables from the Thymio robot, updating its state.

##### 3. Display Thread (<span style="text-decoration: underline;">*display_thread*</span>):

**Function**: display

**Purpose & Execution**: The display function is executed in this thread. It continuously updates the display based on the robot's position, goal, and the environment's state on the screen.

##### 4. Main Thread:

**Function**: __main__

**Purpose**: The main thread that orchestrates the overall control flow.
**Execution: The script starts by initializing various objects, threads, and constants. It then enters the main loop, which handles tasks such as Thymio initialization, path planning, motion updates, obstacle avoidance, and goal-reaching conditions. The main loop is controlled by a timer, and the script continuously monitors the state of the robot and the environment.

##### 5. Notes:

The **Daemon** parameter is always set to True, so when the threads closes when the main program exits.



The main thread orchestrates the overall control flow, handles path planning, motion control, obstacle avoidance, and goal-reaching conditions.




-------
## Vision <a class="anchor" id="computervision"></a>


After trying a lot of days to do shape detections, we decided to use the aruco technic to detect the thymio, the goal.

The obstacles are in black, so the thymio can go everywhere it is white.


#### Map detection<a class="anchor" id="mapdetection"></a>

The map detection is only done once at the begining of the timeline. We first take a single frame, with only the black ostacles and one aruco. The aruco is there so we can compute the grid resolution. It is a calibration image.

The creation of the black and white image involves the is_black_cell function, which evaluates whether a grid cell in the captured image is predominantly black. This function takes an image as input and determines whether the cell contains more than 1/8 black pixels. The process includes converting the image to grayscale, creating a binary mask using a specified threshold, calculating the percentage of black pixels, and comparing it to a threshold percentage (here, set to 10%). If the percentage of black pixels surpasses the threshold, the cell is categorized as black, signifying the presence of obstacles, otherwise, it is classified as white.

In [ ]:
def is_black_cell(image):
    # Know if the grid cell contains more than 1/8 pixels who are black
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(image, 110, 255, cv2.THRESH_BINARY)

    # Calculate the percentage of black pixels in the image
    total_pixels = image.size
    black_pixels = total_pixels - cv2.countNonZero(binary_mask)
    percentage_black = (black_pixels / total_pixels) * 100

    # Set the threshold percentage
    threshold_percentage = 10  # Adjust this threshold as needed

    # Check if the percentage of black pixels exceeds the threshold
    if percentage_black > threshold_percentage:
        return True
    else:
        return False

The purpose is to generate a simplified black and white representation of the environment, highlighting obstacle positions for further processing in Thymio navigation and obstacle avoidance.

The map is created when the following line is called in the main. It will store the builtmap as an image "capture_frame.jpq"

In [ ]:
builtmap, grid_res = apply_grid_to_camera(DEFAULT_GRID_RES)

<div style="text-align: center;">
    <img src="image\captured_frame.png" alt="Setup Map" title="Map" style="width:400px;"/>
</div>


<div style="text-align:center;">
     "captured_frame.png"
</div>

#### Aruco detection<a class="anchor" id="arucodetection"></a>

The Aruco markers are detected using the Aruco marker detection functionality provided by the OpenCV library. The cv2.aruco.getPredefinedDictionary function is used to obtain a predefined Aruco marker dictionary, and cv2.aruco.DetectorParameters is employed to configure the marker detection parameters. The cv2.aruco.detectMarkers function is then applied to find and extract Aruco markers' corners and IDs in the camera frame.

We will use those informations to get the positions of the Thymio and the goal.

In [ ]:
def get_info_arucos(corners, ids, image):

    arucos = []
    if len(corners) > 0:
 
        ids = ids.flatten()
 
        for (markerCorner, markerID) in zip(corners, ids):
            corners = markerCorner.reshape((4, 2))
            (topLeft, topRight, bottomRight, bottomLeft) = corners
 
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))
 
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            
            arucos.append((cX, cY, markerID, (bottomLeft, topLeft)))
            
 
    return image, arucos

#### Thymio detection<a class="anchor" id="thymiodetection"></a>

The ArUco marker 95 is placed on the Thymio. 

The ArUco identification through the *show_robot* function, gives us relevant information such as the center coordinates and orientation of the Thymio.

If the Thymio is not detected (hidden camera for example), it will return a (0,0) position. So the kalman function can continue working without knowing the current localization of the Thymio.

The choice to return this position is due that our reference ArUcO is based at this localisation. So when we are doing the obstacle map, it will be considered such as being an obstacle. The Thymio cannot be at (0,0).

In [21]:
def show_robot(frame, arucos, grid_resolution):
    real_center = robot_center_is(arucos)
    robot_pos = center_in_grid(arucos, 95, grid_resolution)
    angle = get_angle_of_robot(arucos)
    frame = draw_arrow(frame, arucos, angle)

    return frame, arucos, robot_pos, angle

#### Goal detection <a class="anchor" id="goaldetection"></a>

The goal detection process relies on another ArUco marker (n°99) specifically placed to represent the goal location. Similar to Thymio detection, the ArUco marker for the goal is identified in the captured frame. The center coordinates of this marker are extracted to determine the goal's position in the grid. To know its position, we only need to get the center of the ArUcO n°99 by calling this function:

In [15]:
#goal_pos = center_in_grid(arucos, 99, grid_res)

def center_in_grid(arucos, id, grid_resolution):
    if len(arucos) !=0:
        for i in range(len(arucos)):
            if arucos[i][2] == id:
                x = int(arucos[i][0]/grid_resolution)
                y = int(arucos[i][1]/grid_resolution)
                return (x, y)
    return (0, 0)

The variable "last_known_goal_pos" is updated only when the measure goal_position is diffenrent of (0,0). We consider that the goal will not change of position if the camera is hidden. Therefore we are working with the "last_known_goal_pos" variable in the next steps.

#### Obstacles detection <a class="anchor" id="obstaclesdetection"></a>

As explained above, the obstacle detection is based on the assumption that non-white cells in the environment are representing obstacles. 

#### Camera Thread <a class="anchor" id="camerathread"></a>

In the camera thread function *run_camera*, the position of the Thymio and the Goal are only updated when they are none zeros.

Both cannot be located in the (0,0) position in our map, because it is an obstacle. Therefore we know that if we get (0,0) as a position, it means we cannot detect it with the camera.

In [14]:
def run_camera(mes_pos : Robot, mes_goal: Point, camera : Camera, grid_res=DEFAULT_GRID_RES):
    '''
    Function that updates the global Mes_Robot variable with camera data every 0.1 seconds on average

    '''
    cap = cv2.VideoCapture(0)

    ret, frame = cap.read()
    camera.update(ret)
    
    width, height = 707, 10007
    # width, height = set_param_frame(arucos)

    last_known_goal_pos = (0, 0)
    robot_pos = (0, 0)
    while True:
        if cap.isOpened() == False:
            print("Error : video stream closed")
        else:
            frame = cap.read()[1]

            frame, arucos = get_arucos(frame)

            frame, arucos, robot_pos, angle = show_robot(frame, arucos, grid_res)
            goal_pos = center_in_grid(arucos, 99, grid_res)

            if robot_pos != (0,0) :
                robot_pos = Point(robot_pos[0], robot_pos[1])
                mes_pos.update(Robot(robot_pos, angle, True))
            else:
                mes_pos.update(Robot(mes_pos.position, mes_pos.direction, False))
                    
            if goal_pos != (0, 0):
                mes_goal.update(Point(goal_pos[0], goal_pos[1]))
            draw_goal(frame, arucos, grid_res)

            # frame = projected_image(frame, arucos, width, height)
            cv2.imshow("Video Stream", frame)

            # print(f'Robot position: {robot_pos} and angle: {angle}')
            # print(f'Goal position: {goal_pos}')
            
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        time.sleep(0.095)

### Results <a class="anchor" id="results"></a>

The results are in the majority of the time good. Again we mostly have problem with the time it takes the camera to connect to the computer. Therefore we put waiting nodes, to let everything get in place before launching the "race".

To test the vision, you can run the file **test_thread_cam.py". It will show the obstacle map, the localization of the Thymio and the goal.

Press on 'p' to stop the videos, and kill the terminal.

-------

## Path planning<a class="anchor" id="pathplanning"></a>

The goal of the path planning is to find the fastest, so the shortest, path for Thymio to reach the final line. It takes as input the visibility graph of the map, and returns the optimal path, which is a list of coordinates for Santo to follow. 

Where do we get our informations ?

Thanks to the previous steps, we get a global map composed of white and black cells ("capture_frame.jpg") that enables us to know where the thymio can go through.

The start and end positions are defined by the initial position of the Thymio (aruco 95) and the goal (aruco 99) last detected position "last_known_goal_pos".

The Path Planning algorithm used was inspired by the A* algorithm seen in the course excercise session $^{[1]}$.

Path Following Module: the estimation of the robot's position is given by the kalman_filter function.

### Global navigation <a class="anchor" id="globalnavigation"></a>

We decided to us the A* algorithm for the path planning. The goal is to find the optimal path from a starting point to a goal point, considering obstacles in the environment.

Prior computing the optimal path, we had a half thymio length to all obstacle sides, so that the Thymio, considered as a point, want run in it. Now we can calculate the path using the A* algorithm,

#### A* algorithm <a class="anchor" id="Aalgorithm"></a>


We chose to use the A* algorithm because it optimizes the search of the optimal path by expanding the most promising node chosen according some cost and heuristic functions.

<div style="text-align: center;">
    <img src="image\A-algorithm-explanation.png" alt="A algo slide" title="A * algorithm" style="width:400px;"/>
</div>


<div style="text-align:center;">
    "A* algorithm, extending Dijkstra" [1]
</div>

Functions:

$f(n)=g(n)+h(n)$

$f(n)$: The f-score of node nn

$g(n)$: The cost of the cheapest path from the start node to node nn

$h(n)$: The heuristic estimate of the cost from node nn to the goal


The *calculate_path* function orchestrates the entire path planning process presented below:




* **Initialization:**
    The algorithm starts by checking if the start and goal points are within the boundaries of the map and whether they are in a traversable space. If not, an exception is raised.
    The possible movements are defined based on 8-connectivity, meaning the algorithm considers movements in eight possible directions (horizontal, vertical, and diagonal).
    The A* algorithm uses sets (openSet and closedSet) and dictionaries (cameFrom, gScore, and fScore) to keep track of nodes and their associated scores.
    
    
    For node n, cameFrom[n] is the node immediately preceding it on the cheapest path from start to n currently known.

    For node n, gScore[n] is the cost of the cheapest path from start to n currently known.
    
    For node n, fScore[n] := gScore[n] + h(n). map with default value of Infinity

#### Implementation <a class="anchor" id="Aalgorithm"></a>


* **A-Algorithm Implementation:**
    The A* algorithm is implemented using a while loop that continues until there are no more nodes to explore (openSet is empty) or the goal is reached.
    In each iteration, the algorithm selects the node with the lowest fScore from the openSet. This node is set as the current node.
    The neighbors of the current node are explored, and for each neighbor, the algorithm calculates tentative scores (gScore and fScore).
    If a neighbor is not in the openSet, it is added. If the tentative gScore for a neighbor is lower than its current gScore, the algorithm updates the scores and backtracks the optimal path.
    The algorithm continues to explore nodes until the goal is reached or all reachable nodes have been explored.

* **Reconstruction of Path:**
    If the goal is reached, the *reconstruct_path* function is called to reconstruct the optimal path from the start to the goal using the information stored in the cameFrom dictionary.
    The reconstructed path is returned along with the set of nodes that were visited during the algorithm (closedSet).

* **Visualization (Optional):**
    The code includes optional visualization elements, such as displaying the visited nodes with a time delay 0f 0.05 seconds, to visualize the algorithm's progress.

* **Error Handling:**
    If the open set becomes empty and the goal has not been reached, a message is printed indicating that no path was found to the goal.


In [23]:
def calculate_path(env, finalpath, extended_obs, visitedNodes, size_thym, PLOT=False):
    """calls all functions to calculate path
       out: path given in a list of Points 
    """

    grid, start, goal = create_grid(env)
    occupancy_grid = map_without_collision(grid, extended_obs, size_thym)
    max_x, max_y = grid.shape[0], grid.shape[1] # Size of the map
    # List of all coordinates in the grid
    x,y = np.mgrid[0:max_x:1, 0:max_y:1]
    pos = np.empty(x.shape + (2,))
    pos[:, :, 0] = x; pos[:, :, 1] = y
    pos = np.reshape(pos, (x.shape[0]*x.shape[1], 2))
    coords = list([(int(x[0]), int(x[1])) for x in pos])

    # Define the heuristic, here = distance to goal ignoring obstacles
    h = np.linalg.norm(pos - goal, axis=-1)
    h = dict(zip(coords, h))

    # Run the A* algorithm
    path, closedSet = A_Star(start, goal, h, coords, occupancy_grid, visitedNodes)
    path = np.array(path).reshape(-1, 2).transpose()

    closedSet = np.array(closedSet).reshape(-1, 2).transpose()

    convert_path(path, finalpath)

#### Results <a class="anchor" id="A*CCL*"></a>

We are happy with the results of our algorithm. It can very well compute the optimal path through the map. 
The main limit we can see is the time it takes to compute it: if the goal is far away, we need to wait a bit.

<div style="text-align: center;">
    <img src="image\map_path_compute.jpg" alt="pathcomputation" title="Results path computation" style="width:750px;"/> 
</div>


<div style="text-align:center;">
     "Results of the optimal path computation"
</div>

 
The goal is labeled in red, and our thymio has a green arrow on it.


-------


# Kalmann Filtering <a class="anchor" id="filtering"></a>

We decided to work with a Kalman filter because it is optimal for the robot localization due to its ability to handle noise and uncertainties effectively.

Kalman filtering is used to estimate the state of a system based on a series of noisy and incomplete measurements. In the context of robotics and localization, Kalman filters are particularly beneficial for obtaining a good position estimation of a robot. The state of the robot is defined as its position, angle, angular velocity and velocity.

We need to gather informations about the thymio's detected state:
- overhead camera localization
- odometry and thymio's target speed

Furthermore, Kalman filters use a recursive estimation process, meaning that they continually update the state estimate as new measurements become available. This allows the filter to provide real-time position estimates, even if the camera is hidden.

### Mathematical model <a class="anchor" id="mathmodel"></a>

The Thymio's movement is describe by its translation on the x and y axes and its rotation $\theta$. 
It is ruled by the following motion laws :

1. Forward speed :    $v = K*\frac{dmot_R + dmot_L}{2}$
2. Rotational speed : $w = K*\frac{dmot_R - dmot_L}{\pi * THYMIO\_WIDTH}$
3. Position :         $x = v \cdot \cos(\theta) \cdot dt$
4. Position :         $y = v \cdot \sin(\theta) \cdot dt$
5. Direction :        $\theta = w \cdot dt$

(*_K being a conversion factor such that 1 thymio's speed equals K x 1 unit per second (our unit being a grid case))

Our model is thus non linear due to theta !
 

$$X = 
\begin{bmatrix}
v\\
\omega\\
p_x\\
p_y\\
\theta
\end{bmatrix}
$$
Where:
- $v$ is the magnitude speed in $[pxl/s]$.
- $\omega$ is the angular speed $[rad/s]$.
- $p_x$, $p_y$ is the cartesian position in $[pxl]$.
- $\theta$ is the orientation in $[rad]$.


         | 

Our input is $U = [dmot_l, dmot_r]$, the delta between the new and the previous input motor speed.

### Conclusion on filtering <a class="anchor" id="filterconcl"></a>

The extended Kalman filter satifies our needs for this project. It allows to merge the camera measure


-----

## Motion Control <a class="anchor" id="motioncontrol"></a>

To compute the speed of the Thymio, we decided to decompose the computation in three parts: orientation, position and obstacle detection.
The final velocity will be composed of the three different speeds: $\vec{v}_{total}=\vec{v}_{orientation}+\vec{v}_{forward}+\vec{v}_{obstacle}= \begin{bmatrix} v_L \\ v_R \end{bmatrix}$


<div style="text-align:center;">
    <img src="image/motion_control.png" alt="Motion Control" title="A robot is moving around with the proposed motion framework" 
    style="width:300px;"/>
        <img src="image/graph_motion.jpg" alt="Motion Graph" title="Graph motion" style="width:291px;"/>
</div>

<div style="text-align: center;">
   Left: "A robot is moving around with the proposed motion framework" [2]
</div> 


<div style="text-align: center;">
   Right: Angle and major points in our Map
</div> 

### Orientation & angle error <a class="anchor" id="velorientation"> </a>

We define orientation changes as an angle error defined by: $\alpha _E= \alpha _C - \alpha _T$, where $\alpha _T$ is the current angle of the thymio and $\alpha _C$ is the desired angle change.

With classic trigonometry formel we get that: $\alpha _C = arctan(\frac{P_{y+1}-P_{yT}}{P_{x+1}-P_{xT}})$, where $P_T$ the position of the Thymio and $P_{+1}$ the desired position of the Thymio.


One of the big limitations found is because of the use of the angles functions. Indeed, it gives values between $[0; 2\pi]$. So if in some cases we where to near of 0 or $2\pi$, the robot just turns around infinitly. 
Therefore we decided to normalize all angles between $[-\pi; \pi]$, to prevent discontinuities and unexpected behavior.


In [11]:
def compute_angle_error(position, checkpoint, thymio_direction):
  """
  Computes the angle error between the robot and the goal.

  :param position: Current position of the robot.
  :param checkpoint: position to reach (next point in the global path).
  :param thymio_angle: Current orientation of the robot.
  :return: Angle error between the robot and the goal.
  """
  angle = np.arctan2( (checkpoint.y - position.y), (checkpoint.x - position.x))
  # angle = (angle + np.pi) % (2 * np.pi) - np.pi

  thymio_direction = (-thymio_direction + np.pi) % (2 * np.pi) - np.pi    ## + thymio_direction because it's defined in the negative sense of trigo

  angle_error = angle - thymio_direction         
  angle_error = (angle_error + np.pi) % (2 * np.pi) - np.pi

  print("angle: ", angle, "thymio_direction: ", thymio_direction, "angle_error: ", angle_error)

  return angle_error



def compute_derived_angle_error(angle_error, prev_angle_error, dt):
  """
  Computes the derived angle error between the robot and the goal.

  :param angle_error: Angle error between the robot and the goal.
  :param prev_angle_error: Previous angle error between the robot and the goal.
  :param dt: Time step.
  :return: Derived angle error between the robot and the goal.
  """
  return (angle_error - prev_angle_error) / dt

### PD Controler<a class="anchor" id="controler"> </a>

We decided to use a PD controler because of it is simple to implement and it is suitable for fast, real-time feedback.

First, we only had a $K_p$ controler, we could see that the Thymio made some oscillations in its motion, therefore we added the $K_d$ to counteract those undesired movement and so get a more stable response. The derivative helps to reduce the sensitivity of the noise.

$K_{p}$ and $K_d$ are constant.

### Compute velocity <a class="anchor" id="veldirection"> </a>


* *'base_speed'* is the forward velocity that you want the robot to maintain.

$v_{forward}=const.$

The other calculated speeds are going to act as a penalty or gain on top of the *'base_speed'* for each wheel. 

* *'rot_speed'* is the rotational speed calculated using the PD controler based on the angle error (*'angle_error'*) and its derivative (*'derived_angle_error'*).

$\vec{v}_{orientation}=(K_{p} * \alpha _E + K_{d} * \frac{d \alpha _E}{dt} )\begin{bmatrix} 1 \\ -1 \end{bmatrix}$.

The [1,-1] matrix is because to turn, the two wheels need to have opposite speeds.


$\vec{v}_{wheel}= \vec{v}_{forward} + \vec{v}_{orientation}$


The rotational speed is clipped to avoid saturation (>500). If the absolute value of rot_speed exceeds the limit 200 - base_speed, it is adjusted to this limit to prevent oversaturation and the program to crash.



In [10]:
def controller(robot, checkpoint, base_speed, prev_angle_error, dt, final_goal_reached=False):
  '''
  This function computes the motor velocities based on the position, goal, and control parameters.
  param robot : Current position of the robot
  param checkpoint : Goal position to reach (next point in the global path)
  param base_speed : Base speed of the robot
  param final_goal_reached : Flag indicating whether the final goal is reached, not motion is needed.

  return : Input as a tuple (MotL, MotR) 
  return : angle_error
  '''
  if final_goal_reached:
      return 0, 0, 0
  
  angle_error = compute_angle_error(robot.position, checkpoint, robot.direction)
  derived_angle_error = compute_derived_angle_error(angle_error, prev_angle_error, dt)

  rot_speed = Kp * angle_error + Kd * derived_angle_error

  # Clip the rotation speed to avoid saturation
  rot_speed = rot_speed if abs(rot_speed) < 200 - base_speed else 200 - base_speed

  MotL = base_speed + rot_speed
  MotR = base_speed - rot_speed

  return MotL, MotR, angle_error

### Local navigation <a class="anchor" id="localnavigation"> </a>

First we wanted to use the potential field Navigation system and totaly ignoring the other velocities when an obstacle is encountered. It came quickly out that the thymio easily gets out of the desired path. Therefore, we tried to do something similar, but the $\vec{v}_{obstacle}$ will act as a gain on the total computed speed $\vec{v}_{wheel}$. We used the same gains for the proximity sensors as in the exercise session 4 $^{[1]}$.

The external sensors have a stronger weight than the internal ones. The Thymio will be 'repulsed' more quickly when it detects an obstacle, so it will no longer be in line with it.

The code below is similar as the one used in the merge version of the code, to make it work independently.

In [16]:
#if needed decomment:

import tdmclient.notebook
await tdmclient.notebook.start()

In [20]:
%%run_python
timer_period[0] = 10  # 10ms sampling time

@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target
    speed0 = 75     # nominal speed

    # gains used with front proximity sensors [0,5]
    weights = [6, 4, -3, -6, -8]

    addLeft=0
    addRight=0
    diffDelta=0
    # adjustment for obstacles, "gradient" due to obstacles
    for i in range(5):
        addLeft += prox_horizontal[i] * weights[i]//200
        addRight += prox_horizontal[i] * weights[4 - i]//200
    
    motor_left_target = speed0 + addLeft
    motor_right_target = speed0 + addRight

<div style="text-align:center;">
<video width="640" height="360" controls autoplay>
  <source src="image/local_avoidance.mp4" type="video/mp4">
</video>
</div>

### Results <a class="anchor" id="MCresults"> </a>

First we wanted to use 

-----

## Conclusion <a class="anchor" id="conclusion">

To conclude.


### Sources :


[1] MICRO-452 "Basics of mobile robotics" course of EPFL from Professor Francesco Mondada

[2] « Kinematics — CoopRobo 1.0.0 documentation » https://cooprobo.readthedocs.io/en/latest/mobile/pioneer/model/kinematics.html